**Author:** Revekka Gersgovich

**Purpose:** Clean and merge the GSS Data downloaded from GSS website: https://gss.norc.org/get-the-data/stata.html 

**Date:** Nov 29, 2025

In [ ]:
import os
import os.path as path
import pandas as pd
import numpy as np
import glob
import narwhals
import pyreadstat

In [ ]:
parent_dir = os.path.abspath("/Users/revekkagershovich/Documents/Filling_system/Academic/Taste-Based_Discrimination") # Change this directory to run from your computer
assert os.path.exists(parent_dir), "parent_dir does not exist"
os.chdir(parent_dir)

raw_data_dir = path.join(parent_dir, "1_data", "1_raw")
assert os.path.exists(raw_data_dir), "raw_data_dir does not exist"

intermediate_data_dir = path.join(parent_dir, "1_data", "2_intermediate")
assert os.path.exists(intermediate_data_dir), "intermediate_data_dir does not exist"

# Loading Datasets & Saving Metadata

The data was downloaded from GSS Website: https://gss.norc.org/get-the-data/stata.html

It was manually extracted from Zip files prior to loading

In [ ]:
# 1996
df_1996, meta_1996 = pyreadstat.read_dta(
    os.path.join(raw_data_dir, "GSS1996.dta"),
    apply_value_formats=False  # keep numeric codes, don't turn into labels
)

# 2002
df_2002, meta_2002 = pyreadstat.read_dta(
    os.path.join(raw_data_dir, "GSS2002.dta"),
    apply_value_formats=False
)

# 2006
df_2006, meta_2006 = pyreadstat.read_dta(
    os.path.join(raw_data_dir, "GSS2006.dta"),
    apply_value_formats=False
)

# 2018
df_2018, meta_2018 = pyreadstat.read_dta(
    os.path.join(raw_data_dir, "GSS2018.dta"),
    apply_value_formats=False
)

# 2024 (inside subfolder)
df_2024, meta_2024 = pyreadstat.read_dta(
    os.path.join(raw_data_dir, "2024", "GSS2024.dta"),
    apply_value_formats=False
)

In [ ]:
def save_metadata_to_csv(meta, year):
    labels_df = pd.DataFrame({
        "variable": meta.column_names,
        "label": meta.column_labels
    })
    labels_df.to_csv(os.path.join(raw_data_dir, f"GSS_{year}_variable_labels.csv"), index=False)

save_metadata_to_csv(meta_1996, 1996)
save_metadata_to_csv(meta_2002, 2002)
save_metadata_to_csv(meta_2006, 2006)
save_metadata_to_csv(meta_2018, 2018)
save_metadata_to_csv(meta_2024, 2024)

In [ ]:
df_1996.shape, df_2002.shape,df_2006.shape, df_2018.shape, df_2024.shape

# Merging Datasets

## Check variable consistency across years

In [ ]:
set_1996 = set(df_1996.columns)
set_2002 = set(df_2002.columns)
set_2006 = set(df_2006.columns)
set_2018 = set(df_2018.columns)
set_2024 = set(df_2024.columns)

datasets = {
    "1996": set_1996,
    "2002": set_2002,
    "2006": set_2006,
    "2018": set_2018,
    "2024": set_2024
}

In [ ]:
# Define list of years with adult mental health stigma module, and years with child mental health stigma module
adult_years = ["1996", "2006", "2018", "2024"]
child_years = ["2002", "2024"]

# Adult stigma variables: common to 1996, 2006, 2018, 2024
adult_stigma_vars = set.intersection(*(datasets[year] for year in adult_years))

# Child stigma variables: common to 2002 and 2024
child_stigma_vars = set.intersection(*(datasets[year] for year in child_years))

# Respondent mental health variables in various years
mh_vars = [
    "evbrkdwn",   # ever felt like having a nervous breakdown
    "relmhsp1",   # patient was self (mental health help-seeking)
    "evmhp",      # ever had a mental health problem
    "mntlhlth",   # days of poor mental health, past 30 days
    "govmentl",   # attitudes about government and mental health
    "depress",    # ever told by a doctor you had depression
    "diagnosd",   # ever diagnosed with mental health problem
    "mhtreatd",   # ever treated for mental health problem
    "emoprobs"    # emotional problems interfering with life
]

In [ ]:
# 1. Union of all variables you care about
keep_vars = adult_stigma_vars | child_stigma_vars | set(mh_vars)

# (Optional but very useful): also keep 'year' if it exists
keep_vars_with_year = set(keep_vars) | {"year"}

# 2. Helper to subset a df safely
def subset_wave(df, year_label):
    cols_in_df = set(df.columns)
    cols_to_keep = list(keep_vars_with_year & cols_in_df)

    tmp = df[cols_to_keep].copy()

    # Ensure a proper 'year' column exists
    if "year" not in tmp.columns:
        tmp["year"] = int(year_label)

    return tmp

# 3. Apply to each wave
df_1996_sub = subset_wave(df_1996, 1996)
df_2002_sub = subset_wave(df_2002, 2002)
df_2006_sub = subset_wave(df_2006, 2006)
df_2018_sub = subset_wave(df_2018, 2018)
df_2024_sub = subset_wave(df_2024, 2024)

# 4. Concatenate all waves into a single dataset
df = pd.concat(
    [df_1996_sub, df_2002_sub, df_2006_sub, df_2018_sub, df_2024_sub],
    ignore_index=True,
    sort=False
)

# Validate

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['year'].value_counts()

In [ ]:
df.sample(5, random_state=42)

In [ ]:
df.columns

# Saving Datasets and Metadata

In [ ]:
# Put metadata objects into a dict for iteration
meta_dict = {
    "1996": meta_1996,
    "2002": meta_2002,
    "2006": meta_2006,
    "2018": meta_2018,
    "2024": meta_2024
}

# Start the unified codebook using *all* variables from the final df
unified_vars = set(df.columns)
unified_codebook = pd.DataFrame({"variable": sorted(unified_vars)})

# For each year, merge in that year's labels
for year, meta in meta_dict.items():
    year_cb = pd.DataFrame({
        "variable": meta.column_names,
        f"label_{year}": meta.column_labels
    })
    unified_codebook = unified_codebook.merge(year_cb, on="variable", how="left")

# Add a convenience column showing all years where the variable appears
def list_years_present(row):
    present = [year for year in meta_dict.keys()
               if pd.notna(row[f"label_{year}"])]
    return ", ".join(present)

unified_codebook["years_present"] = unified_codebook.apply(list_years_present, axis=1)

# Save it
unified_codebook.to_csv(
    os.path.join(intermediate_data_dir, "codebook.csv"),
    index=False
)

In [ ]:
unified_codebook.head()

In [ ]:
unified_codebook['years_present'].value_counts()

In [ ]:
mh_child_codebook = unified_codebook[unified_codebook['years_present'] ==  "2002, 2024"]
mh_child_codebook
mh_child_codebook.to_csv(
    os.path.join(intermediate_data_dir, "mh_child_codebook.csv"),
    index=False
)

In [ ]:
mh_adult_codebook = unified_codebook[unified_codebook['years_present'] ==  "1996, 2006, 2018, 2024"]
mh_adult_codebook

mh_adult_codebook.to_csv(
    os.path.join(intermediate_data_dir, "mh_adult_codebook.csv"),
    index=False
)


In [ ]:
df.to_csv(os.path.join(intermediate_data_dir, "gss_cleaned_96_02_06_18_24.csv"), index=False)